# Análise Automatizada de TCC em Administração## Rotinas de Processamento de Linguagem Natural e Clusterização**Autores:** [Seu Nome]  **Instituição:** PUCPR  **Data:** Dezembro 2024---### Sobre este notebookEste notebook contém as rotinas completas de análise automatizada de conteúdo utilizadas no artigo "Análise Automatizada da Evolução dos Trabalhos de Conclusão em Administração: Uma Abordagem com Modelos de Linguagem de Grande Escala".**Objetivos:**- Pré-processar textos de TCC (normalização, tokenização, lematização)- Vetorizar textos com TF-IDF- Aplicar clusterização K-Means- Gerar visualizações (nuvens de palavras, heatmaps)- Identificar termos emergentes e declinantes**Requisitos:**- Python 3.10+- Bibliotecas listadas em `requirements.txt`- Arquivo de dados: `dados_tcc_anonimizados.csv`---

## 1. Instalação de DependênciasExecute esta célula apenas na primeira vez ou em ambiente novo (Google Colab, etc.).

In [ ]:
# Instalar bibliotecas necessárias!pip install pandas numpy nltk spacy scikit-learn matplotlib seaborn wordcloud openpyxl# Baixar recursos do NLTKimport nltknltk.download('stopwords')nltk.download('punkt')nltk.download('rslp')# Baixar modelo spaCy para português!python -m spacy download pt_core_news_sm

## 2. Importação de Bibliotecas

In [ ]:
# Manipulação de dadosimport pandas as pdimport numpy as np# Processamento de textoimport reimport nltkfrom nltk.corpus import stopwordsfrom nltk.tokenize import word_tokenizefrom nltk.stem import RSLPStemmerimport spacy# Machine Learningfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.cluster import KMeansfrom sklearn.decomposition import PCAfrom sklearn.metrics import silhouette_score# Visualizaçãoimport matplotlib.pyplot as pltimport seaborn as snsfrom wordcloud import WordCloud# Configuraçõesimport warningswarnings.filterwarnings('ignore')# Configurar estilo de gráficosplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")print("✓ Bibliotecas importadas com sucesso!")

## 3. Carregamento e Preparação dos Dados**Nota:** O arquivo `dados_tcc_anonimizados.csv` deve estar no mesmo diretório do notebook ou você deve ajustar o caminho abaixo.

In [ ]:
# Carregar dadosdf = pd.read_csv('dados_tcc_anonimizados.csv', encoding='utf-8')# Visualizar estruturaprint(f"Dimensões do dataset: {df.shape}")print(f"\nColunas disponíveis:\n{df.columns.tolist()}")print(f"\nPrimeiras linhas:")df.head()

In [ ]:
# Verificar dados ausentesprint("Dados ausentes por coluna:")print(df.isnull().sum())# Estatísticas descritivas básicasprint("\n" + "="*50)print("ESTATÍSTICAS DESCRITIVAS")print("="*50)print(f"Período: {df['ano'].min()} - {df['ano'].max()}")print(f"Total de TCC: {len(df)}")print(f"\nDistribuição por ano:")print(df['ano'].value_counts().sort_index())

## 4. Pré-processamento de TextoFunções para normalização, limpeza e preparação dos textos.

In [ ]:
# Carregar recursos de PLNstop_words = set(stopwords.words('portuguese'))# Adicionar stopwords customizadas (específicas do domínio)stop_words_custom = {    'empresa', 'negócio', 'plano', 'trabalho', 'projeto',     'tcc', 'administração', 'ano', 'mês', 'dia'}stop_words.update(stop_words_custom)# Carregar modelo spaCynlp = spacy.load('pt_core_news_sm', disable=['parser', 'ner'])print(f"✓ Stopwords carregadas: {len(stop_words)} termos")print(f"✓ Modelo spaCy carregado: {nlp.meta['name']}")

In [ ]:
def preprocessar_texto(texto, lematizar=False, min_len=3):    """    Pré-processa texto para análise de PLN.    Parâmetros:    -----------    texto : str        Texto a ser processado    lematizar : bool, default=False        Se True, aplica lematização (recomendado para textos longos)    min_len : int, default=3        Comprimento mínimo de tokens a manter    Retorna:    --------    str        Texto processado    """    # Verificar se texto é válido    if pd.isna(texto) or not isinstance(texto, str):        return ""    # 1. Normalização    texto = texto.lower()    # 2. Remover URLs, emails e caracteres especiais    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto)    texto = re.sub(r'\S+@\S+', '', texto)    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', texto)    # 3. Remover espaços múltiplos    texto = re.sub(r'\s+', ' ', texto).strip()    # 4. Tokenização    tokens = word_tokenize(texto, language='portuguese')    # 5. Filtrar tokens    tokens = [        t for t in tokens         if t not in stop_words and len(t) >= min_len    ]    # 6. Lematização (opcional)    if lematizar and tokens:        doc = nlp(' '.join(tokens))        tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words]    return ' '.join(tokens)# Testar funçãotexto_exemplo = "A empresa de tecnologia desenvolveu um aplicativo inovador para gestão de projetos."texto_processado = preprocessar_texto(texto_exemplo, lematizar=True)print(f"Original: {texto_exemplo}")print(f"Processado: {texto_processado}")

In [ ]:
# Aplicar pré-processamento ao dataset# Escolher campo textual principal (ajustar conforme sua base)campo_texto = 'clusters_tematizados'  # ou 'resumo', 'abstract', etc.print(f"Processando campo: {campo_texto}")print("Isso pode levar alguns minutos...")df['texto_processado'] = df[campo_texto].apply(    lambda x: preprocessar_texto(x, lematizar=True))# Remover documentos vazios após processamentodf_limpo = df[df['texto_processado'].str.len() > 0].copy()print(f"\n✓ Pré-processamento concluído!")print(f"  Documentos originais: {len(df)}")print(f"  Documentos válidos: {len(df_limpo)}")print(f"  Documentos removidos: {len(df) - len(df_limpo)}")# Atualizar dataframedf = df_limpo

## 5. Vetorização TF-IDFTransformação dos textos em representação numérica.

In [ ]:
# Configurar vetorizadorvectorizer = TfidfVectorizer(    max_features=2000,        # Limitar vocabulário aos 2000 termos mais relevantes    ngram_range=(1, 2),       # Considerar unigramas e bigramas    min_df=2,                 # Termo deve aparecer em pelo menos 2 documentos    max_df=0.8,               # Termo não pode aparecer em mais de 80% dos documentos    sublinear_tf=True,        # Aplicar escala logarítmica ao TF    norm='l2'                 # Normalização L2)# Ajustar e transformarprint("Vetorizando textos...")tfidf_matrix = vectorizer.fit_transform(df['texto_processado'])# Obter vocabuláriofeature_names = vectorizer.get_feature_names_out()print(f"\n✓ Vetorização concluída!")print(f"  Dimensões da matriz: {tfidf_matrix.shape}")print(f"  Documentos: {tfidf_matrix.shape[0]}")print(f"  Termos no vocabulário: {tfidf_matrix.shape[1]}")print(f"  Esparsidade: {(1 - tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])):.2%}")

In [ ]:
# Visualizar termos mais frequentes no corpussoma_tfidf = np.array(tfidf_matrix.sum(axis=0)).flatten()termos_freq = sorted(zip(feature_names, soma_tfidf), key=lambda x: x[1], reverse=True)print("\nTop 30 termos mais relevantes (TF-IDF):")print("-" * 50)for i, (termo, peso) in enumerate(termos_freq[:30], 1):    print(f"{i:2d}. {termo:25s} {peso:8.2f}")

## 6. Clusterização K-MeansIdentificação de grupos temáticos nos TCC.

In [ ]:
# Testar diferentes valores de Kprint("Testando diferentes números de clusters...")K_range = range(3, 11)inertias = []silhouette_scores = []for k in K_range:    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)    labels = kmeans.fit_predict(tfidf_matrix)    inertias.append(kmeans.inertia_)    silhouette_scores.append(silhouette_score(tfidf_matrix, labels))    print(f"  K={k}: Inércia={kmeans.inertia_:.2f}, Silhueta={silhouette_scores[-1]:.3f}")print("\n✓ Testes concluídos!")

In [ ]:
# Visualizar métricas de avaliaçãofig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))# Método do cotoveloax1.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)ax1.set_xlabel('Número de Clusters (K)', fontsize=12)ax1.set_ylabel('Inércia (Within-Cluster Sum of Squares)', fontsize=12)ax1.set_title('Método do Cotovelo', fontsize=14, fontweight='bold')ax1.grid(True, alpha=0.3)ax1.set_xticks(K_range)# Coeficiente de Silhuetaax2.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)ax2.set_xlabel('Número de Clusters (K)', fontsize=12)ax2.set_ylabel('Coeficiente de Silhueta', fontsize=12)ax2.set_title('Coeficiente de Silhueta', fontsize=14, fontweight='bold')ax2.grid(True, alpha=0.3)ax2.set_xticks(K_range)ax2.axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Limiar razoável')ax2.legend()plt.tight_layout()plt.savefig('metricas_clustering.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Gráfico salvo: metricas_clustering.png")

In [ ]:
# Aplicar K-Means com K=7 (escolhido após análise)K_FINAL = 7print(f"Aplicando K-Means com K={K_FINAL}...")kmeans_final = KMeans(n_clusters=K_FINAL, random_state=42, n_init=10, max_iter=300)df['cluster'] = kmeans_final.fit_predict(tfidf_matrix)# Estatísticas dos clustersprint(f"\n✓ Clusterização concluída!")print(f"\nDistribuição de documentos por cluster:")print("-" * 50)cluster_counts = df['cluster'].value_counts().sort_index()for cluster_id, count in cluster_counts.items():    pct = (count / len(df)) * 100    print(f"  Cluster {cluster_id}: {count:3d} documentos ({pct:5.1f}%)")

## 7. Análise e Interpretação dos ClustersExtração dos termos mais relevantes de cada cluster.

In [ ]:
def extrair_termos_cluster(cluster_id, n_termos=15):    """    Extrai os termos mais relevantes de um cluster.    Parâmetros:    -----------    cluster_id : int        ID do cluster a analisar    n_termos : int, default=15        Número de termos a extrair    Retorna:    --------    list        Lista de tuplas (termo, peso_tfidf)    """    # Filtrar documentos do cluster    indices = df[df['cluster'] == cluster_id].index    # Calcular média TF-IDF para cada termo no cluster    cluster_tfidf = tfidf_matrix[indices].mean(axis=0).A1    # Ordenar termos por peso    top_indices = cluster_tfidf.argsort()[-n_termos:][::-1]    top_terms = [(feature_names[i], cluster_tfidf[i]) for i in top_indices]    return top_terms# Extrair e exibir termos de todos os clustersprint("TERMOS MAIS RELEVANTES POR CLUSTER")print("=" * 70)for cluster_id in range(K_FINAL):    n_docs = len(df[df['cluster'] == cluster_id])    print(f"\n{'='*70}")    print(f"CLUSTER {cluster_id} ({n_docs} documentos)")    print('='*70)    termos = extrair_termos_cluster(cluster_id, n_termos=15)    for i, (termo, peso) in enumerate(termos, 1):        print(f"  {i:2d}. {termo:30s} {peso:.4f}")

In [ ]:
# Criar DataFrame com resumo dos clusterscluster_summary = []for cluster_id in range(K_FINAL):    termos = extrair_termos_cluster(cluster_id, n_termos=10)    termos_str = ', '.join([t[0] for t in termos[:5]])    cluster_summary.append({        'Cluster': cluster_id,        'N_Documentos': len(df[df['cluster'] == cluster_id]),        'Principais_Termos': termos_str    })df_clusters = pd.DataFrame(cluster_summary)print("\nResumo dos Clusters:")print(df_clusters.to_string(index=False))# Salvar resumodf_clusters.to_csv('resumo_clusters.csv', index=False, encoding='utf-8')print("\n✓ Resumo salvo: resumo_clusters.csv")

## 8. Visualização dos Clusters (PCA)Redução de dimensionalidade para visualização em 2D.

In [ ]:
# Aplicar PCA para reduzir para 2 dimensõesprint("Aplicando PCA...")pca = PCA(n_components=2, random_state=42)coords_2d = pca.fit_transform(tfidf_matrix.toarray())df['pca_x'] = coords_2d[:, 0]df['pca_y'] = coords_2d[:, 1]print(f"✓ PCA concluído!")print(f"  Variância explicada: {pca.explained_variance_ratio_.sum():.2%}")print(f"    PC1: {pca.explained_variance_ratio_[0]:.2%}")print(f"    PC2: {pca.explained_variance_ratio_[1]:.2%}")

In [ ]:
# Plotar clusters em 2Dplt.figure(figsize=(14, 10))# Cores para cada clustercolors = plt.cm.tab10(np.linspace(0, 1, K_FINAL))for cluster_id in range(K_FINAL):    cluster_data = df[df['cluster'] == cluster_id]    plt.scatter(        cluster_data['pca_x'],        cluster_data['pca_y'],        c=[colors[cluster_id]],        label=f'Cluster {cluster_id} (n={len(cluster_data)})',        alpha=0.6,        s=50,        edgecolors='black',        linewidth=0.5    )plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} da variância)', fontsize=12)plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} da variância)', fontsize=12)plt.title('Visualização dos Clusters Temáticos (PCA)', fontsize=14, fontweight='bold')plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)plt.grid(True, alpha=0.3)plt.tight_layout()plt.savefig('clusters_pca.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Gráfico salvo: clusters_pca.png")

## 9. Nuvens de Palavras por PeríodoAnálise da evolução temporal dos termos.

In [ ]:
def gerar_nuvem_palavras(textos, titulo, max_words=100, figsize=(15, 7)):    """    Gera nuvem de palavras a partir de lista de textos.    Parâmetros:    -----------    textos : list        Lista de textos processados    titulo : str        Título do gráfico    max_words : int, default=100        Número máximo de palavras na nuvem    figsize : tuple, default=(15, 7)        Tamanho da figura    """    # Concatenar todos os textos    texto_completo = ' '.join(textos)    if not texto_completo.strip():        print(f"Aviso: Nenhum texto disponível para '{titulo}'")        return    # Gerar nuvem    wordcloud = WordCloud(        width=1600,        height=800,        background_color='white',        max_words=max_words,        colormap='viridis',        relative_scaling=0.5,        min_font_size=10,        collocations=False    ).generate(texto_completo)    # Plotar    plt.figure(figsize=figsize)    plt.imshow(wordcloud, interpolation='bilinear')    plt.axis('off')    plt.title(titulo, fontsize=16, fontweight='bold', pad=20)    plt.tight_layout(pad=0)    # Salvar    filename = titulo.lower().replace(' ', '_').replace('-', '_') + '.png'    plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor='white')    plt.show()    print(f"✓ Nuvem salva: {filename}")print("Gerando nuvens de palavras por período...")

In [ ]:
# Definir períodosperiodos = {    '2013-2016': (2013, 2016),    '2017-2020': (2017, 2020),    '2021-2024': (2021, 2024)}# Gerar nuvem para cada períodofor nome_periodo, (ano_ini, ano_fim) in periodos.items():    dados_periodo = df[(df['ano'] >= ano_ini) & (df['ano'] <= ano_fim)]    textos = dados_periodo['texto_processado'].dropna().tolist()    print(f"\nPeríodo {nome_periodo}: {len(textos)} documentos")    gerar_nuvem_palavras(        textos,         f'Nuvem de Palavras - {nome_periodo}',        max_words=80    )

## 10. Análise de Tendências: Termos Emergentes e DeclinantesIdentificação de termos com maior variação de frequência ao longo do tempo.

In [ ]:
def analisar_tendencias_termos(df, campo_ano='ano', campo_texto='texto_processado', n_termos=20):    """    Identifica termos emergentes e declinantes comparando dois períodos.    Parâmetros:    -----------    df : DataFrame        DataFrame com os dados    campo_ano : str        Nome da coluna com o ano    campo_texto : str        Nome da coluna com texto processado    n_termos : int        Número de termos a retornar em cada categoria    Retorna:    --------    tuple        (emergentes, declinantes) - listas de tuplas (termo, variação)    """    # Dividir em dois períodos (antes e depois da mediana)    ano_mediano = df[campo_ano].median()    periodo1 = df[df[campo_ano] <= ano_mediano][campo_texto].dropna()    periodo2 = df[df[campo_ano] > ano_mediano][campo_texto].dropna()    print(f"Período 1: {len(periodo1)} documentos (até {int(ano_mediano)})")    print(f"Período 2: {len(periodo2)} documentos (após {int(ano_mediano)})")    # Vetorizar cada período    vec1 = TfidfVectorizer(max_features=500, ngram_range=(1, 2), min_df=2)    vec2 = TfidfVectorizer(max_features=500, ngram_range=(1, 2), min_df=2)    tfidf1 = vec1.fit_transform(periodo1)    tfidf2 = vec2.fit_transform(periodo2)    # Calcular frequências médias    freq1 = dict(zip(vec1.get_feature_names_out(), tfidf1.mean(axis=0).A1))    freq2 = dict(zip(vec2.get_feature_names_out(), tfidf2.mean(axis=0).A1))    # Calcular variações para termos comuns    termos_comuns = set(freq1.keys()) & set(freq2.keys())    variacoes = {        termo: (freq2[termo] - freq1[termo], freq1[termo], freq2[termo])        for termo in termos_comuns    }    # Ordenar por variação absoluta    emergentes = sorted(        variacoes.items(),         key=lambda x: x[1][0],         reverse=True    )[:n_termos]    declinantes = sorted(        variacoes.items(),         key=lambda x: x[1][0]    )[:n_termos]    return emergentes, declinantes# Executar análiseprint("Analisando tendências temporais...")print("=" * 70)emergentes, declinantes = analisar_tendencias_termos(df, n_termos=20)

In [ ]:
# Exibir resultadosprint("\n" + "=" * 70)print("TERMOS EMERGENTES (maior crescimento)")print("=" * 70)print(f"{'#':<4} {'Termo':<30} {'Variação':<12} {'Período 1':<12} {'Período 2':<12}")print("-" * 70)for i, (termo, (var, freq1, freq2)) in enumerate(emergentes, 1):    print(f"{i:<4} {termo:<30} {var:>+11.4f} {freq1:>11.4f} {freq2:>11.4f}")print("\n" + "=" * 70)print("TERMOS DECLINANTES (maior queda)")print("=" * 70)print(f"{'#':<4} {'Termo':<30} {'Variação':<12} {'Período 1':<12} {'Período 2':<12}")print("-" * 70)for i, (termo, (var, freq1, freq2)) in enumerate(declinantes, 1):    print(f"{i:<4} {termo:<30} {var:>+11.4f} {freq1:>11.4f} {freq2:>11.4f}")

In [ ]:
# Visualizar tendênciasfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))# Termos emergentestermos_emerg = [t[0] for t in emergentes[:15]]vars_emerg = [t[1][0] for t in emergentes[:15]]ax1.barh(range(len(termos_emerg)), vars_emerg, color='green', alpha=0.7)ax1.set_yticks(range(len(termos_emerg)))ax1.set_yticklabels(termos_emerg)ax1.set_xlabel('Variação TF-IDF', fontsize=11)ax1.set_title('Top 15 Termos Emergentes', fontsize=13, fontweight='bold')ax1.grid(axis='x', alpha=0.3)ax1.invert_yaxis()# Termos declinantestermos_decl = [t[0] for t in declinantes[:15]]vars_decl = [t[1][0] for t in declinantes[:15]]ax2.barh(range(len(termos_decl)), vars_decl, color='red', alpha=0.7)ax2.set_yticks(range(len(termos_decl)))ax2.set_yticklabels(termos_decl)ax2.set_xlabel('Variação TF-IDF', fontsize=11)ax2.set_title('Top 15 Termos Declinantes', fontsize=13, fontweight='bold')ax2.grid(axis='x', alpha=0.3)ax2.invert_yaxis()plt.tight_layout()plt.savefig('tendencias_termos.png', dpi=300, bbox_inches='tight')plt.show()print("\n✓ Gráfico salvo: tendencias_termos.png")

## 11. Associação entre Clusters e Áreas SetoriaisAnálise da relação entre temas e setores de mercado.

In [ ]:
# Verificar se existe coluna de área setorialif 'area_setorial' in df.columns:    # Criar tabela de contingência normalizada por coluna    contingencia = pd.crosstab(        df['cluster'],         df['area_setorial'],        normalize='columns'    )    # Plotar heatmap    plt.figure(figsize=(14, 9))    sns.heatmap(        contingencia,        annot=True,        fmt='.1%',        cmap='YlOrRd',        cbar_kws={'label': 'Proporção'},        linewidths=0.5,        linecolor='gray'    )    plt.title('Associação entre Clusters Temáticos e Áreas Setoriais',               fontsize=14, fontweight='bold', pad=20)    plt.xlabel('Área Setorial', fontsize=12)    plt.ylabel('Cluster', fontsize=12)    plt.xticks(rotation=45, ha='right')    plt.yticks(rotation=0)    plt.tight_layout()    plt.savefig('heatmap_cluster_setor.png', dpi=300, bbox_inches='tight')    plt.show()    print("✓ Heatmap salvo: heatmap_cluster_setor.png")    # Identificar associações fortes    print("\n" + "=" * 70)    print("ASSOCIAÇÕES FORTES (proporção > 30%)")    print("=" * 70)    associacoes_fortes = []    for cluster in contingencia.index:        for setor in contingencia.columns:            valor = contingencia.loc[cluster, setor]            if valor > 0.30:                associacoes_fortes.append((cluster, setor, valor))                print(f"  Cluster {cluster} × {setor}: {valor:.1%}")    if not associacoes_fortes:        print("  Nenhuma associação forte identificada (limiar: 30%)")else:    print("Aviso: Coluna 'area_setorial' não encontrada no dataset.")    print("Pulando análise de associação cluster × setor.")

## 12. Exportação de ResultadosSalvar dados processados e resultados das análises.

In [ ]:
# Preparar dataset final com clustersdf_export = df[[    'id_tcc', 'ano', 'titulo', 'cluster',     'texto_processado', 'pca_x', 'pca_y']].copy()# Adicionar rótulos descritivos dos clusters (ajustar conforme interpretação)rotulos_clusters = {    0: "Cluster 0",    1: "Cluster 1",    2: "Cluster 2",    3: "Cluster 3",    4: "Cluster 4",    5: "Cluster 5",    6: "Cluster 6"}df_export['cluster_label'] = df_export['cluster'].map(rotulos_clusters)# Salvardf_export.to_csv('dados_com_clusters.csv', index=False, encoding='utf-8')print("✓ Dataset com clusters salvo: dados_com_clusters.csv")# Salvar termos por clusterwith open('termos_por_cluster.txt', 'w', encoding='utf-8') as f:    f.write("TERMOS MAIS RELEVANTES POR CLUSTER\n")    f.write("=" * 70 + "\n\n")    for cluster_id in range(K_FINAL):        n_docs = len(df[df['cluster'] == cluster_id])        f.write(f"\nCLUSTER {cluster_id} ({n_docs} documentos)\n")        f.write("-" * 70 + "\n")        termos = extrair_termos_cluster(cluster_id, n_termos=20)        for i, (termo, peso) in enumerate(termos, 1):            f.write(f"  {i:2d}. {termo:30s} {peso:.4f}\n")print("✓ Termos por cluster salvos: termos_por_cluster.txt")# Salvar tendênciasdf_tendencias = pd.DataFrame({    'Termo': [t[0] for t in emergentes],    'Variacao': [t[1][0] for t in emergentes],    'Freq_Periodo1': [t[1][1] for t in emergentes],    'Freq_Periodo2': [t[1][2] for t in emergentes],    'Tipo': 'Emergente'})df_tendencias = pd.concat([    df_tendencias,    pd.DataFrame({        'Termo': [t[0] for t in declinantes],        'Variacao': [t[1][0] for t in declinantes],        'Freq_Periodo1': [t[1][1] for t in declinantes],        'Freq_Periodo2': [t[1][2] for t in declinantes],        'Tipo': 'Declinante'    })])df_tendencias.to_csv('tendencias_termos.csv', index=False, encoding='utf-8')print("✓ Tendências de termos salvas: tendencias_termos.csv")print("\n" + "=" * 70)print("ANÁLISE CONCLUÍDA!")print("=" * 70)print("\nArquivos gerados:")print("  • dados_com_clusters.csv")print("  • resumo_clusters.csv")print("  • termos_por_cluster.txt")print("  • tendencias_termos.csv")print("  • metricas_clustering.png")print("  • clusters_pca.png")print("  • heatmap_cluster_setor.png")print("  • tendencias_termos.png")print("  • nuvem_de_palavras_*.png (por período)")

---## Informações Adicionais### CitaçãoSe você utilizar este código, por favor cite:```[Seu Nome]. (2024). Análise Automatizada de TCC em Administração: Rotinas de PLN e Clusterização. GitHub. https://github.com/seu-usuario/seu-repo```### LicençaEste código está disponível sob licença MIT.### ContatoPara dúvidas ou sugestões: [seu-email@instituicao.edu.br]---**Última atualização:** Dezembro 2024